# Lib import

In [45]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import seaborn as sns

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d  
from matplotlib import cm

# Загрузка данных и просмотр их первоначальной структуры

Загружаем датасет с GitHub:

In [46]:
import zipfile
import os

!wget --no-check-certificate \
    "https://github.com/PolMix/nems_ai/archive/refs/heads/main.zip" \
    -O "/tmp/data.zip"


zip_ref = zipfile.ZipFile('/tmp/data.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2023-05-13 11:15:22--  https://github.com/PolMix/nems_ai/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/PolMix/nems_ai/zip/refs/heads/main [following]
--2023-05-13 11:15:23--  https://codeload.github.com/PolMix/nems_ai/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/data.zip’

/tmp/data.zip           [      <=>           ]  12.69M  7.74MB/s    in 1.6s    

2023-05-13 11:15:24 (7.74 MB/s) - ‘/tmp/data.zip’ saved [13310665]



In [47]:
%cd /tmp/nems_ai-main/Data
filename = 'Dataset.txt'
df = pd.read_fwf(filename)

/tmp/nems_ai-main/Data


"Читаем" датасет:

In [48]:
df.head()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),Pretension (Pa),Eigenfrequency (Hz),Quality factor (1),Effective mass,Noise (kg^2 / s^3),TED (W)
0,10.666186,759.454713,192.911166,46.497924,19.089801,465.822644,28.323179,9.003567e+08,6.415102308860925E7+2400.623397410659i,1.336133e+04,2.362616e-15,1.051171e-22,2.602258e-12
1,10.666186,759.454713,192.911166,46.497924,19.089801,465.822644,28.323179,9.003567e+08,1.6011837483039114E8+6638.939975583381i,1.205903e+04,2.589984e-15,3.186781e-22,2.991534e-12
2,10.666186,759.454713,192.911166,46.497924,19.089801,465.822644,28.323179,9.003567e+08,2.909667084628352E8+12724.292499287933i,1.143351e+04,2.560363e-15,6.037978e-22,3.409430e-12
3,10.666186,759.454713,192.911166,46.497924,19.089801,465.822644,28.323179,9.003567e+08,3.96451977228644E8+19548.92557221195i,1.013999e+04,8.780197e-16,3.181145e-22,4.959458e-12
4,105.195552,890.696186,209.252268,37.407275,3.716553,314.636835,30.305477,5.090849e+08,2569354.395333146+0.17898817427683494i,7.177442e+06,3.216064e-14,2.135344e-28,7.976759e-12


# Функции для обработки датасета

Функция, реализующая трансформацию датасета 4row x 1col --> 1row x 4col:

In [49]:
def process_data(df, len_common):
    # num_common - количество стоблцов, одинаковых для всех мод
    data = []
    for index in range(df.shape[0]):
        if index % 4 == 0:
            i = 0
            row = df.iloc[index].values.flatten().tolist()
        else:
            row += df.iloc[index, len_common:].values.flatten().tolist()
        i += 1
        if i == 4:
            data.append(row)
    return data

Функция, реализующая замену индексовых имен колонок на буквенные:

In [50]:
def name_data_columns(df, data, len_common):
    '''
    Names columns in data DataFrame.
    - df - initial DataFrame of 4row1col format that has named columns
    - data - new DataFrame of 1row4col format which columns are to be named
    - len_common - number of columns that are common for all resonant modes
    '''
    len_differ = df.shape[1] - len_common # число различающихся колонок

    cols_common = list(df.columns)[:len_common]
    cols_differ = list(df.columns)[len_common:]
    print(len_differ, len(cols_differ))

    columns = [] # сюда будем записывать финальный вариант колонок.
    # Прим.: pandas не дает изменять колонки внутри цикла,
    # их можно изменить только единственным присваиванием, 
    # именно поэтому сначала все append-им в один список
    
    # проименовываем общие колонки:
    for col_index in range(0, len_common):
        columns.append(cols_common[col_index])
    
    # проименовываем различающиеся колонки:
    for mode_index in range(1, 5):
        for col_index in range(0, len_differ):
            columns.append(f'M{mode_index} ' + cols_differ[col_index])

    # Присваиваем имена колонок
    data.columns = columns

    return data

Функция, возвращающая индексы строк (и столбцов, если рассматривать разные моды колебаний), в которых детектирована аномалия $ Im\left(f_0\right) < 0$:

In [51]:
def return_neg_frequencies_index(df, freq_indices):
    '''
    Returns indices of the objects having negative values of 
    imaginary part of resonant frequency which results in negative quality factor.
    - df - The DataFrame to be checked
    - freq_indices - indices of resonant frequencies in df
    '''
    list_i = []
    list_j = []
    for j in freq_indices:
        for i in range(df.shape[0]):
            if '-' in df.iloc[i,j]:
                list_i.append(i)
                list_j.append(j)
    return list_i, list_j

Функция, реализующая удаление описанных выше аномалий:

In [52]:
def del_neg_frequencies(df, freq_indices):
    '''
    Deletes objects that have negative imaginary part of resonant frequency.
    - df - DataFrame to be processed
    - freq_indices - indices of resonant frequencies in df
    '''
    lst_i, _ = return_neg_frequencies_index(data, freq_indices)
    lst_i = [x for x in set(lst_i)]
    lst_i.sort(reverse=True)
    for i in lst_i:
        df.drop(index=i, inplace=True)
    return df

Функция, реализующая удаление мнимой части резонансной частоты для всех элементов датасета:

In [53]:
def del_im_frequency(df, freq_indices):
    for j in freq_indices:
        for i in range(0, df.shape[0]):
            df.iloc[i, j] = float(str(df.iloc[i, j]).split('+', 1)[0])
    return df

Функция, реализующая округление чисел до `decimals` знаков после запятой (потому что метод конечных элементов иногда, например, вместо числа `200` записывает `200.0000000111`):

In [54]:
def round_data_float(data, decimals):
    return data.round(decimals=decimals)

Функция, реализующая округление значений до типа `int`, которые должны были быть записаны методом конечных элементов как `int`, но были записаны как `float`:

In [55]:
def round_data_int(data, cols_indices_to_round):
    for col_index in cols_indices_to_round:
        data.iloc[:, col_index] = data.iloc[:, col_index].astype(int)

# Обработка датасета

Преобразование 4row x 1col --> 1row x 4col:

In [56]:
data = pd.DataFrame(process_data(df, len_common=8))
data.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,10.666186,759.454713,192.911166,46.497924,19.089801,465.822644,28.323179,9.003567e+08,6.415102308860925E7+2400.623397410659i,1.336133e+04,...,2.909667084628352E8+12724.292499287933i,1.143351e+04,2.560363e-15,6.037978e-22,3.409430e-12,3.96451977228644E8+19548.92557221195i,1.013999e+04,8.780197e-16,3.181145e-22,4.959458e-12
1,105.195552,890.696186,209.252268,37.407275,3.716553,314.636835,30.305477,5.090849e+08,2569354.395333146+0.17898817427683494i,7.177442e+06,...,8224138.945881522+0.7163319699598331i,5.740452e+06,3.281744e-14,8.720428e-28,1.367533e-11,1.1522259804500692E7+1.1514823841560045i,5.003229e+06,3.291971e-14,1.406151e-27,1.583936e-11
2,92.051879,213.886957,78.837771,23.255635,0.017793,266.965778,0.000000,9.811963e+08,3534070.7381136324+0.014998583641082577i,1.178135e+08,...,1.0633838931726312E7+0.048525947723057454i,1.095686e+08,2.955338e-15,3.326099e-32,9.503560e-13,1.4215267434397759E7-1.7026735618501154i,4.174396e+06,2.955390e-15,1.167079e-30,1.919962e-12
3,10.310316,221.113532,195.903798,37.079461,0.027053,251.500693,26.675832,6.351363e+08,3.3531096619837992E7+6.265918625790593i,2.675673e+06,...,1.2676424894250964E8+33.51392465609216i,1.891218e+06,7.206070e-16,6.788349e-28,2.254001e-12,1.9236968865737534E8+56.254152580472244i,1.709827e+06,7.147747e-16,1.130223e-27,2.477637e-12
4,18.683573,282.662136,207.614117,18.750307,0.496423,172.245797,2.647445,0.000000e+00,1.0490776466649445E7+0.5979256153162141i,8.772643e+06,...,4.475570525582115E7+2.685519572702211i,8.332783e+06,1.623836e-15,6.849769e-28,2.943955e-12,7.057575996260074E7+4.410434723934928i,8.000998e+06,1.616668e-15,1.119973e-27,3.081568e-12


Форматирование названий колонок:

In [57]:
data = name_data_columns(df, data, len_common=8)
data.head()

5 5


,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),Pretension (Pa),M1 Eigenfrequency (Hz),M1 Quality factor (1),...,M3 Eigenfrequency (Hz),M3 Quality factor (1),M3 Effective mass,M3 Noise (kg^2 / s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor (1),M4 Effective mass,M4 Noise (kg^2 / s^3),M4 TED (W)
0,10.666186,759.454713,192.911166,46.497924,19.089801,465.822644,28.323179,9.003567e+08,6.415102308860925E7+2400.623397410659i,1.336133e+04,...,2.909667084628352E8+12724.292499287933i,1.143351e+04,2.560363e-15,6.037978e-22,3.409430e-12,3.96451977228644E8+19548.92557221195i,1.013999e+04,8.780197e-16,3.181145e-22,4.959458e-12
1,105.195552,890.696186,209.252268,37.407275,3.716553,314.636835,30.305477,5.090849e+08,2569354.395333146+0.17898817427683494i,7.177442e+06,...,8224138.945881522+0.7163319699598331i,5.740452e+06,3.281744e-14,8.720428e-28,1.367533e-11,1.1522259804500692E7+1.1514823841560045i,5.003229e+06,3.291971e-14,1.406151e-27,1.583936e-11
2,92.051879,213.886957,78.837771,23.255635,0.017793,266.965778,0.000000,9.811963e+08,3534070.7381136324+0.014998583641082577i,1.178135e+08,...,1.0633838931726312E7+0.048525947723057454i,1.095686e+08,2.955338e-15,3.326099e-32,9.503560e-13,1.4215267434397759E7-1.7026735618501154i,4.174396e+06,2.955390e-15,1.167079e-30,1.919962e-12
3,10.310316,221.113532,195.903798,37.079461,0.027053,251.500693,26.675832,6.351363e+08,3.3531096619837992E7+6.265918625790593i,2.675673e+06,...,1.2676424894250964E8+33.51392465609216i,1.891218e+06,7.206070e-16,6.788349e-28,2.254001e-12,1.9236968865737534E8+56.254152580472244i,1.709827e+06,7.147747e-16,1.130223e-27,2.477637e-12
4,18.683573,282.662136,207.614117,18.750307,0.496423,172.245797,2.647445,0.000000e+00,1.0490776466649445E7+0.5979256153162141i,8.772643e+06,...,4.475570525582115E7+2.685519572702211i,8.332783e+06,1.623836e-15,6.849769e-28,2.943955e-12,7.057575996260074E7+4.410434723934928i,8.000998e+06,1.616668e-15,1.119973e-27,3.081568e-12


Вывод информации об аномалиях мнимой части резонансной частоты, их удаление:

In [58]:
frequency_indices = [8, 13, 18, 23] # индексы колонок, в которых содержатся значения резонансных частот

# Детектирование аномалий:
lst_i, lst_j = return_neg_frequencies_index(data, frequency_indices)
print("Row and column indices of detected anomalies of imaginary part of resonant frequency:")
print(lst_i, lst_j)
print(f"The length of the dataset is {data.shape[0]}")

# Удаление аномалий:
data = del_neg_frequencies(data, frequency_indices)
lst_i, lst_j = return_neg_frequencies_index(data, frequency_indices)
print(f"Anomalies have been deleted, the length of the dataset is {data.shape[0]}")

Row and column indices of detected anomalies of imaginary part of resonant frequency:
[128, 176, 193, 198, 268, 301, 325, 343, 345, 350, 431, 463, 468, 476, 496, 498, 503, 507, 529, 592, 593, 630, 682, 692, 744, 755, 808, 827, 832, 833, 865, 894, 896, 972, 1007, 1052, 1080, 1217, 1313, 1315, 1322, 1336, 1346, 1367, 1380, 1406, 1476, 1491, 1529, 1539, 1545, 1603, 1610, 1611, 1613, 1652, 1663, 1673, 1693, 1721, 1729, 1743, 1789, 1824, 1830, 1844, 1845, 1871, 1877, 1956, 1984, 2071, 2081, 2083, 2100, 2115, 2124, 2181, 2189, 2277, 2312, 2316, 2343, 2360, 2411, 2430, 2506, 2544, 2598, 2654, 2888, 2903, 2909, 2947, 2990, 3078, 3111, 3232, 3252, 3362, 3421, 3539, 3555, 3569, 3642, 3655, 3673, 3694, 3701, 3715, 3797, 3898, 4013, 4027, 4075, 4079, 4159, 4185, 4220, 4236, 4240, 4313, 4319, 4323, 4336, 4393, 4410, 4434, 4447, 4487, 4504, 4510, 4520, 4529, 4537, 4542, 4584, 4611, 4633, 4667, 4701, 4752, 4840, 4893, 4948, 4953, 5000, 5095, 5101, 5140, 5153, 5243, 5277, 5283, 5287, 5297, 128, 193, 1

Удаление мнимой части всех элементов датасета:

In [59]:
data = del_im_frequency(data, [8, 13, 18, 23])
data.head()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),Pretension (Pa),M1 Eigenfrequency (Hz),M1 Quality factor (1),...,M3 Eigenfrequency (Hz),M3 Quality factor (1),M3 Effective mass,M3 Noise (kg^2 / s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor (1),M4 Effective mass,M4 Noise (kg^2 / s^3),M4 TED (W)
0,10.666186,759.454713,192.911166,46.497924,19.089801,465.822644,28.323179,9.003567e+08,64151023.088609,1.336133e+04,...,290966708.462835,1.143351e+04,2.560363e-15,6.037978e-22,3.409430e-12,396451977.228644,1.013999e+04,8.780197e-16,3.181145e-22,4.959458e-12
1,105.195552,890.696186,209.252268,37.407275,3.716553,314.636835,30.305477,5.090849e+08,2569354.395333,7.177442e+06,...,8224138.945882,5.740452e+06,3.281744e-14,8.720428e-28,1.367533e-11,11522259.804501,5.003229e+06,3.291971e-14,1.406151e-27,1.583936e-11
3,10.310316,221.113532,195.903798,37.079461,0.027053,251.500693,26.675832,6.351363e+08,33531096.619838,2.675673e+06,...,126764248.94251,1.891218e+06,7.206070e-16,6.788349e-28,2.254001e-12,192369688.657375,1.709827e+06,7.147747e-16,1.130223e-27,2.477637e-12
4,18.683573,282.662136,207.614117,18.750307,0.496423,172.245797,2.647445,0.000000e+00,10490776.466649,8.772643e+06,...,44755705.255821,8.332783e+06,1.623836e-15,6.849769e-28,2.943955e-12,70575759.962601,8.000998e+06,1.616668e-15,1.119973e-27,3.081568e-12
5,121.747842,991.550766,102.506945,29.297485,2.037235,415.471216,18.419812,8.374213e+08,2649738.745159,7.314592e+06,...,8269130.657285,6.082613e+06,2.259615e-14,2.331664e-28,2.518650e-11,11381108.823312,5.405428e+06,2.267905e-14,3.624442e-28,2.982471e-11


Дополнительно удалим строку с индексом `2`, потому что в ячейке `[2, 'M1 TED (W)']` содержится аномалия в данных (неопознанное вторжение символа `s` в `float`-значение) (поймано на графиках распределений):

In [60]:
#data = data.drop(2)
#print(df.iloc[0:3, 17])

In [61]:
data.describe()

,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),Pretension (Pa),M1 Quality factor (1),M1 Effective mass,...,M2 Noise (kg^2 / s^3),M2 TED (W),M3 Quality factor (1),M3 Effective mass,M3 Noise (kg^2 / s^3),M3 TED (W),M4 Quality factor (1),M4 Effective mass,M4 Noise (kg^2 / s^3),M4 TED (W)
count,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4.909000e+03,4.909000e+03,4.909000e+03,...,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03
mean,195.469415,529.097516,150.819856,30.404340,23.742353,349.708484,17.227796,4.712346e+08,2.502643e+07,2.791775e-14,...,1.034791e-23,5.672434e-12,2.219908e+07,2.808115e-14,2.014207e-23,7.943504e-12,2.115728e+07,2.806848e-14,1.691331e-23,1.017113e-11
std,238.666890,269.281493,41.982835,11.712403,49.683629,145.560699,12.786909,3.190863e+08,8.329148e+07,4.391482e-14,...,2.047638e-22,5.494667e-12,7.344745e+07,4.388424e-14,3.843014e-22,7.561086e-12,8.666440e+07,4.391170e-14,2.545723e-22,9.704030e-12
min,10.006357,50.114701,66.096716,10.055810,0.010004,100.222269,0.000000,0.000000e+00,1.409866e+03,1.256945e-16,...,5.865251e-34,1.153546e-14,1.409853e+03,1.263636e-16,8.801085e-34,3.347321e-14,1.409842e+03,1.266484e-16,9.123322e-34,7.378453e-14
25%,36.605352,298.766024,107.242205,20.117549,0.309281,220.216016,5.140889,2.366180e+08,2.943045e+05,3.492639e-15,...,6.930178e-30,1.884886e-12,2.537637e+05,3.638416e-15,1.125043e-29,2.898015e-12,2.296537e+05,3.614451e-15,1.998218e-29,3.819665e-12
50%,97.149246,526.461810,150.537470,30.610468,2.657895,349.938323,17.005210,4.777992e+08,1.872628e+06,1.032800e-14,...,5.438185e-28,4.082376e-12,1.620164e+06,1.061271e-14,9.200261e-28,5.521390e-12,1.356222e+06,1.062533e-14,1.503564e-27,7.042181e-12
75%,257.772109,758.297463,193.682337,40.815722,19.179475,476.299101,28.608417,7.477595e+08,1.221293e+07,3.247702e-14,...,3.186328e-26,7.780666e-12,1.071371e+07,3.272321e-14,5.545136e-26,1.055391e-11,9.048577e+06,3.280904e-14,8.259448e-26,1.334680e-11
max,1195.787212,999.773967,234.883842,49.992126,299.880837,599.954225,39.995581,9.994946e+08,1.467439e+09,4.326970e-13,...,1.062324e-20,6.391638e-11,1.207506e+09,4.327311e-13,1.952995e-20,7.927833e-11,2.734598e+09,4.327028e-13,9.359624e-21,1.112834e-10


Задаем ограничения на значения параметров:

In [62]:
def catch_parameter_overflow(df, param_name, lower_limit, upper_limit):
    mask = (df[param_name].astype(float) > lower_limit) & (df[param_name].astype(float) < upper_limit)
    return df[mask]

def catch_overflow(df, param_limits):

    # Это кусок кода на случай, если на рез. частоту ограничений не накладывается, 
    #   но в любом случае надо конвертировать данные в тип float
    if 'Eigenfrequency (Hz)' not in param_limits.keys():
        for mode_number in range(1, 5):
            df.loc[:, f'M{mode_number} ' + 'Eigenfrequency (Hz)'] = df.loc[:, f'M{mode_number} ' + 'Eigenfrequency (Hz)'].astype(float)

    for param in param_limits.keys():
        for mode_number in range(1, 5):

            # Преобразуем тип резонансной частоты в тип float
            if param == 'Eigenfrequency (Hz)':
                df.loc[:, f'M{mode_number} ' + param] = df.loc[:, f'M{mode_number} ' + param].astype(float)

            # Последовательно накладываем маски ограничений
            df = catch_parameter_overflow(df, param_name=f'M{mode_number} ' + param,
                                          lower_limit=param_limits[param][0],
                                          upper_limit=param_limits[param][1])
    return df

In [63]:
param_limits = {#'Eigenfrequency (Hz)': [0, 3e7], 
                #'Quality factor': [0, 5e8], 
                #'TED (W)': [1e-20, 1]
                }

print('Dataset length before anomaly check is {length}'.format(length=data.shape[0]))
data_catched = catch_overflow(data, param_limits)
print('Dataset length after anomaly check is {length}'.format(length=data_catched.shape[0]))
data_catched.describe()

Dataset length before anomaly check is 4909
Dataset length after anomaly check is 4909


<ipython-input-62-3b1675c7008f>:11: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, f'M{mode_number} ' + 'Eigenfrequency (Hz)'] = df.loc[:, f'M{mode_number} ' + 'Eigenfrequency (Hz)'].astype(float)


,Beam length (um),Beam width (nm),Thickness_1 (nm),Thickness_2 (nm),Temperature (K),Distance (nm),Gate voltage (V),Pretension (Pa),M1 Eigenfrequency (Hz),M1 Quality factor (1),...,M3 Eigenfrequency (Hz),M3 Quality factor (1),M3 Effective mass,M3 Noise (kg^2 / s^3),M3 TED (W),M4 Eigenfrequency (Hz),M4 Quality factor (1),M4 Effective mass,M4 Noise (kg^2 / s^3),M4 TED (W)
count,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4.909000e+03,4.909000e+03,4.909000e+03,...,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03,4.909000e+03
mean,195.469415,529.097516,150.819856,30.404340,23.742353,349.708484,17.227796,4.712346e+08,6.988914e+06,2.502643e+07,...,2.804685e+07,2.219908e+07,2.808115e-14,2.014207e-23,7.943504e-12,4.173375e+07,2.115728e+07,2.806848e-14,1.691331e-23,1.017113e-11
std,238.666890,269.281493,41.982835,11.712403,49.683629,145.560699,12.786909,3.190863e+08,1.098401e+07,8.329148e+07,...,5.009151e+07,7.344745e+07,4.388424e-14,3.843014e-22,7.561086e-12,7.418109e+07,8.666440e+07,4.391170e-14,2.545723e-22,9.704030e-12
min,10.006357,50.114701,66.096716,10.055810,0.010004,100.222269,0.000000,0.000000e+00,3.840254e+04,1.409866e+03,...,1.153109e+05,1.409853e+03,1.263636e-16,8.801085e-34,3.347321e-14,1.538710e+05,1.409842e+03,1.266484e-16,9.123322e-34,7.378453e-14
25%,36.605352,298.766024,107.242205,20.117549,0.309281,220.216016,5.140889,2.366180e+08,8.730700e+05,2.943045e+05,...,2.650417e+06,2.537637e+05,3.638416e-15,1.125043e-29,2.898015e-12,3.557429e+06,2.296537e+05,3.614451e-15,1.998218e-29,3.819665e-12
50%,97.149246,526.461810,150.537470,30.610468,2.657895,349.938323,17.005210,4.777992e+08,2.470389e+06,1.872628e+06,...,7.744980e+06,1.620164e+06,1.061271e-14,9.200261e-28,5.521390e-12,1.059702e+07,1.356222e+06,1.062533e-14,1.503564e-27,7.042181e-12
75%,257.772109,758.297463,193.682337,40.815722,19.179475,476.299101,28.608417,7.477595e+08,7.668586e+06,1.221293e+07,...,2.693589e+07,1.071371e+07,3.272321e-14,5.545136e-26,1.055391e-11,3.932619e+07,9.048577e+06,3.280904e-14,8.259448e-26,1.334680e-11
max,1195.787212,999.773967,234.883842,49.992126,299.880837,599.954225,39.995581,9.994946e+08,8.115660e+07,1.467439e+09,...,3.675392e+08,1.207506e+09,4.327311e-13,1.952995e-20,7.927833e-11,4.399336e+08,2.734598e+09,4.327028e-13,9.359624e-21,1.112834e-10


Дополнительно: еще раз изменим тип всех данных на `float`:

In [64]:
for param in list(data_catched.columns)[:8]:
    data_catched.loc[:, param] = data_catched.loc[:, param].astype(float)

for row_index in range(data_catched.shape[0]):
    if data_catched.loc[data_catched.index[row_index], 'Gate voltage (V)'] == '5.630590021610321E':
        print(row_index)

for param in list(data_catched.columns)[8:]:
    data_catched.loc[:, param] = data_catched.loc[:, param].astype(float)

# Сохраним датасет на диск:

In [65]:
data_catched.columns = ['Beam length (um)', 'Beam width (nm)', 'Thickness_1 (nm)', 'Thickness_2 (nm)', 
                        'Temperature (K)', 'Distance (nm)', 'Gate voltage (V)', 'Pretension (Pa)',
                        'M1 Eigenfrequency (Hz)', 'M1 Quality factor', 'M1 Effective mass (kg)', 'M1 TED (W)', 'M1 Noise (kg^2/s^3)',
                        'M2 Eigenfrequency (Hz)', 'M2 Quality factor', 'M2 Effective mass (kg)', 'M2 TED (W)', 'M2 Noise (kg^2/s^3)',
                        'M3 Eigenfrequency (Hz)', 'M3 Quality factor', 'M3 Effective mass (kg)', 'M3 TED (W)', 'M3 Noise (kg^2/s^3)',
                        'M4 Eigenfrequency (Hz)', 'M4 Quality factor', 'M4 Effective mass (kg)', 'M4 TED (W)', 'M4 Noise (kg^2/s^3)']

In [66]:
data_catched.to_csv('Dataset_Processed.csv')

# Добавление вклада термоупругих потерь

Подмешаем к добротности в датасете добротность согласно приближению Roukes:

In [22]:
# Fundamental properties:
rho = [3100, 2700] # density of matter
E = [250e9, 70e9] # Young's modulus
k = [20, 237] # Thermal conductivity constant
alpha = [2.3e-6, 23.1e-6] # Thermal expansion coefficient

# Calculated properties:
Cp = [700, 904] # heat capacity at constant pressure

In [23]:
def calculate_Q_TED(df):
    param_history = {'Cp_av': [], 'E_av': [], 'rho_av': [], 'k_av': [], 'alpha_av': [], 'xi_av': [], 'chi': [], 'Q_ted': []}
    for row_index in range(0, df.shape[0]):
        # Calculating average properties of beam:
        Cp_av = (Cp[0] * df.iloc[row_index, 2] + Cp[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        E_av = (E[0] * df.iloc[row_index, 2] + E[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        rho_av = (rho[0] * df.iloc[row_index, 2] + rho[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        k_av = (k[0] * df.iloc[row_index, 2] + k[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        alpha_av = (alpha[0] * df.iloc[row_index, 2] + alpha[1] * df.iloc[row_index, 3]) / (df.iloc[row_index, 2] + df.iloc[row_index, 3])
        xi_av = k_av / (rho_av * Cp_av)
        param_history['Cp_av'].append(Cp_av)
        param_history['E_av'].append(E_av)
        param_history['rho_av'].append(rho_av)
        param_history['k_av'].append(k_av)
        param_history['alpha_av'].append(alpha_av)
        param_history['xi_av'].append(xi_av)

        for mode_number in range(1, 5):
            # Calculating TED quality factor:
            chi = df.iloc[row_index, 1] * 1e-9 * np.sqrt(2 * 3.1415 * df.loc[df.index[row_index], f'M{mode_number} Eigenfrequency (Hz)'] / (2 * xi_av))
            delta_ted = (E_av * alpha_av **2 * df.iloc[row_index, 4]) / (Cp_av * rho_av) * (6/(chi**2) - 6 * (np.sinh(chi) + np.sin(chi))/(chi**3 * (np.cosh(chi) + np.cos(chi))))
            df.loc[df.index[row_index], f'M{mode_number} Quality factor'] = 1/(1/df.loc[df.index[row_index], f'M{mode_number} Quality factor'] + delta_ted)

            param_history['chi'].append(chi)
            param_history['Q_ted'].append(1/delta_ted)
        
    return df, param_history

In [24]:
print(data_catched.shape[0])
data_ted, history = calculate_Q_TED(data_catched)

4909


KeyError: ignored

In [ ]:
sorted_history = sorted(history['Q_ted'])
print(sorted_history)

In [ ]:
data_ted.head()